# Import statements

In [130]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'


In [131]:
#Reading the data
df = pd.read_csv("C:/Users/ayu_a/Documents/Untitled Folder/concated.csv")

In [70]:
#Droping unwanted columns
df.drop(['HotelRatings','HotelAddress','HotelName','Unnamed: 0','Authorlocation','AuthorName'],axis=1,inplace=True)

In [71]:
#checking the null values
(df.isnull().sum(axis=0)/len(df))*100

HotelPrice                                   0.000000
Review                                       0.000000
title                                        0.000000
Service                                      9.054203
Cleanliness                                  9.181028
Overall                                      0.000000
Value                                        9.029691
Sleep Quality                               47.947068
Rooms                                       17.773840
Location                                    22.571790
Business service (e.g., internet access)    89.300052
Check in / front desk                       82.523484
dtype: float64

In [72]:
#Droping  columns having nulls with high percentage
df.drop(['Sleep Quality','Business service (e.g., internet access)','Check in / front desk'],axis=1,inplace=True)

In [77]:
df.head()

,HotelPrice,Review,title,Service,Cleanliness,Overall,Value,Rooms,Location
0,$135 - $193*,"Usually stay near the airport, but this trip w...",“Great place and location”,5.0,5.0,5.0,5.0,5.0,5.0
1,$135 - $193*,Stayed at this Hilton for 2 nights. It was lik...,"“Nice stay, nice surroundings”",4.0,4.0,4.0,4.0,4.0,5.0
2,$135 - $193*,"Stayed there one night, December 16, on the wa...",“Perfect for an overnight just off I-95”,5.0,5.0,4.0,3.0,5.0,5.0
3,$135 - $193*,I just stayed here last weekend and have alrea...,“Great Hotel”,5.0,5.0,5.0,4.0,5.0,5.0
4,$135 - $193*,My mother who is 90 and I stayed one night on ...,“good room for handicapped person”,5.0,5.0,5.0,4.0,5.0,5.0


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476839 entries, 0 to 1476838
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   HotelPrice   1476839 non-null  object 
 1   Review       1476839 non-null  object 
 2   title        1476839 non-null  object 
 3   Service      1343123 non-null  float64
 4   Cleanliness  1341250 non-null  float64
 5   Overall      1476839 non-null  float64
 6   Value        1343485 non-null  float64
 7   Rooms        1214348 non-null  float64
 8   Location     1143490 non-null  float64
dtypes: float64(6), object(3)
memory usage: 101.4+ MB


In [75]:
df.Overall.value_counts()

5.0    615476
4.0    463650
3.0    195121
2.0    103418
1.0     99173
0.0         1
Name: Overall, dtype: int64

In [81]:
df["ReviewDetails"]=df['title']+" "+df["Review"]

In [82]:
df

,HotelPrice,Review,title,Service,Cleanliness,Overall,Value,Rooms,Location,ReviewDetails
0,$135 - $193*,"Usually stay near the airport, but this trip w...",“Great place and location”,5.0,5.0,5.0,5.0,5.0,5.0,“Great place and location” Usually stay near t...
1,$135 - $193*,Stayed at this Hilton for 2 nights. It was lik...,"“Nice stay, nice surroundings”",4.0,4.0,4.0,4.0,4.0,5.0,"“Nice stay, nice surroundings” Stayed at this ..."
2,$135 - $193*,"Stayed there one night, December 16, on the wa...",“Perfect for an overnight just off I-95”,5.0,5.0,4.0,3.0,5.0,5.0,“Perfect for an overnight just off I-95” Staye...
3,$135 - $193*,I just stayed here last weekend and have alrea...,“Great Hotel”,5.0,5.0,5.0,4.0,5.0,5.0,“Great Hotel” I just stayed here last weekend ...
4,$135 - $193*,My mother who is 90 and I stayed one night on ...,“good room for handicapped person”,5.0,5.0,5.0,4.0,5.0,5.0,“good room for handicapped person” My mother w...
...,...,...,...,...,...,...,...,...,...,...
1476834,$184 - $341*,I just visited NYC this past weekend with fami...,“Awesome Location. Nice rooms!”,NaN,NaN,4.0,NaN,NaN,NaN,“Awesome Location. Nice rooms!” I just visited...
1476835,$184 - $341*,Location is alright. A good walk from Times Sq...,“its alright”,NaN,NaN,3.0,NaN,NaN,NaN,“its alright” Location is alright. A good walk...
1476836,$184 - $341*,We just got back last weekend. We were looking...,"“Fabulous for the price, nice and clean!! Perf...",3.0,3.0,4.0,4.0,3.0,NaN,"“Fabulous for the price, nice and clean!! Perf..."
1476837,$184 - $341*,Ok... while most of the reviews seem good of t...,“Nightmare!!”,NaN,NaN,1.0,NaN,NaN,NaN,“Nightmare!!” Ok... while most of the reviews ...


In [128]:
def print_plot(index):
    example = df[df.index == index][['ReviewDetails' ,'Overall']].values[0]
    if len(example) > 0:
        print(example[0])
        print('rating:', example[1])
        print('price',example[2])

In [129]:
print_plot(10)

great eperience due unforeseen circumstances found looking hotel area remembered hotels white marsh area immediately called hilton quoted single king bed  asked military discount room came  wow great surprise especially since  high got definitely saw hotel higher prices great hotel room fridge microwave nice coffee area great sitting area couch desk bed bed great molds body amazing rough day nice comfortable place loved also offered free internet need would stay would def recommend town guests btw front desk girl friendly seemed genuinely enjoy hotel job
rating: 5.0
price $135 - $193*


In [85]:
print_plot(100)

“Hotel Monaco - great Seattle location” For location, this hotel is ideal - it is right in the heart of downtown, we walked to Pioneer Sq, Pike Street market, downtown shopping etc. The 5pm wine reception is a really nice touch. Check-in was a bit odd - they seemed not to find our booking and were a bit uncommunicative, then our room was 904, next to the elevators, which we heard constantly. Not a big deal, other then early in the morning but one to avoid. Overall, good for a short stay.
rating: 4.0


### Stopword Removal and data cleaning

In [89]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['ReviewDetails'] = df['ReviewDetails'].apply(clean_text)

In [95]:
df['ReviewDetails'] = df['ReviewDetails'].str.replace('\d+', '')

In [108]:
print_plot(1)

nice stay nice surroundings stayed hilton  nights like hilton garden inns nice people pleasant cheap good breakfast  small nice workout facilities pool good rooms bathrooms appreciated free shuttle mall plenty restaurants ikea etc
rating: 4.0


In [98]:
print_plot(100)

hotel monaco great seattle location location hotel ideal right heart downtown walked pioneer sq pike street market downtown shopping etc pm wine reception really nice touch checkin bit odd seemed find booking bit uncommunicative room  net elevators heard constantly big deal early morning one avoid overall good short stay
rating: 4.0


#### Tokenizing and one hot encoding

In [99]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['ReviewDetails'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 550808 unique tokens.


In [100]:
# Transform each text in texts in a sequence of integers.
X = tokenizer.texts_to_sequences(df['ReviewDetails'].values)
#pad_sequences is used to ensure that all sequences in a list have the same length. 
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (1476839, 250)


In [105]:
#Convert categorical variable into dummy/indicator variables.
Y = pd.get_dummies(df['Overall'],drop_first=True).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (1476839, 5)


#### Train and Test split

In [110]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1329155, 250) (1329155, 5)
(147684, 250) (147684, 5)


# LSTM Model Training 

In [113]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 505       
Total params: 5,080,905
Trainable params: 5,080,905
Non-trainable params: 0
_________________________________________________________________
None


## Batch processing

In [114]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

C:\Users\ayu_a\.conda\envs\workspace-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 1196239 samples, validate on 132916 samples
Epoch 1/5
1196239/1196239 [==============================] - 7689s 6ms/step - loss: 0.7576 - accuracy: 0.6660 - val_loss: 0.6897 - val_accuracy: 0.6940
Epoch 2/5
1196239/1196239 [==============================] - 10664s 9ms/step - loss: 0.6696 - accuracy: 0.7072 - val_loss: 0.6690 - val_accuracy: 0.7051
Epoch 3/5
1196239/1196239 [==============================] - 6701s 6ms/step - loss: 0.6308 - accuracy: 0.7282 - val_loss: 0.6608 - val_accuracy: 0.7133
Epoch 4/5
1196239/1196239 [==============================] - 7868s 7ms/step - loss: 0.5987 - accuracy: 0.7454 - val_loss: 0.6571 - val_accuracy: 0.7176
Epoch 5/5
1196239/1196239 [==============================] - 5961s 5ms/step - loss: 0.5718 - accuracy: 0.7595 - val_loss: 0.6546 - val_accuracy: 0.7223


In [119]:
model.predict(X_test)

array([[1.0245628e-04, 1.7513325e-04, 2.4444382e-03, 1.8806244e-01,
        8.0921549e-01],
       [9.5818996e-01, 3.8782384e-02, 2.8560306e-03, 1.2588214e-04,
        4.5685458e-05],
       [9.2310955e-05, 3.2390954e-04, 4.2756936e-03, 3.9902568e-01,
        5.9628236e-01],
       ...,
       [1.2598938e-04, 2.2746113e-03, 6.3549519e-02, 8.9569688e-01,
        3.8352940e-02],
       [1.1080383e-02, 2.9962507e-01, 4.6426031e-01, 2.1918403e-01,
        5.8502164e-03],
       [9.2724877e-05, 8.9268846e-04, 6.0167465e-02, 7.6645440e-01,
        1.7239277e-01]], dtype=float32)

# Model Evaluation

In [115]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

147684/147684 [==============================] - 809s 5ms/step
Test set
  Loss: 0.650
  Accuracy: 0.726


# Test with a new review.

In [125]:
new_review = ['The room was dump; there was a small wardrobe which we could not use as it was not pleasant (to me). I will have rather place a open wardrobe.']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = [1,2,3,4,5]
print(pred, labels[np.argmax(pred)])

[[0.15679988 0.46004072 0.36422518 0.01243879 0.00649538]] 2
